In [ ]:
# importa todo y define una función de ejecución de pruebas
from importlib import reload
from ayudante import ejecutar_prueba

import bloomfilter
import bloque
import cce
import ayudante
import bloquemerkle
import network
import script
import tx

In [ ]:
# Ejemplo de Bloom Filter

from ayudante import doble_sha256

tamaño_campo_bit = 10
campo_bit = [0] * tamaño_campo_bit

h = doble_sha256(b'hello world')
bit = int.from_bytes(h, 'big') % tamaño_campo_bit
campo_bit[bit] = 1
print(campo_bit)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [ ]:
# Ejemplo de Bloom Filter 2

from ayudante import doble_sha256

tamaño_campo_bit = 10
campo_bit = [0] * tamaño_campo_bit

h = doble_sha256(b'hello world')
bit = int.from_bytes(h, 'big') % tamaño_campo_bit
campo_bit[bit] = 1
h = doble_sha256(b'goodbye')
bit = int.from_bytes(h, 'big') % tamaño_campo_bit
campo_bit[bit] = 1
print(campo_bit)

[0, 0, 1, 0, 0, 0, 0, 0, 0, 1]


In [ ]:
# Ejemplo Bloom Filter 3

from ayudante import doble_sha256, hash160

tamaño_campo_bit = 10
campo_bit = [0] * tamaño_campo_bit

frase1 = b'hello world'
h1 = doble_sha256(frase1)
bit1 = int.from_bytes(h1, 'big') % tamaño_campo_bit
campo_bit[bit1] = 1
h2 = hash160(frase1)
bit2 = int.from_bytes(h2, 'big') % tamaño_campo_bit
campo_bit[bit2] = 1
frase2 = b'goodbye'
h1 = doble_sha256(frase2)
bit1 = int.from_bytes(h1, 'big') % tamaño_campo_bit
campo_bit[bit1] = 1
h2 = hash160(frase2)
bit2 = int.from_bytes(h2, 'big') % tamaño_campo_bit
campo_bit[bit2] = 1
print(campo_bit)

[1, 1, 1, 0, 0, 0, 0, 0, 0, 1]


In [ ]:
# Ejemplo BIP0037 Bloom Filter

from ayudante import murmur3
from bloomfilter import CONSTANTE_BIP37

tamaño_campo = 2
num_funciones = 2
ajuste = 42

tamaño_campo_bit = tamaño_campo * 8
campo_bit = [0] * tamaño_campo_bit

for frase in (b'hello world', b'goodbye'):
    for i in range(num_funciones):
        semilla = i * CONSTANTE_BIP37 + ajuste
        h = murmur3(frase, semilla=semilla)
        bit = h % tamaño_campo_bit
        campo_bit[bit] = 1
print(campo_bit)

[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0]


### Ejercicio 1

#### 1.1 Dado un Bloom Filter con estos parámetros: tamaño=10, función contador=5, ajuste=99, qué bits tenemos tras añadir estas cosas? 

 * `b'Hello World'`
 * `b'Goodbye!'`

#### 1.2. Haz que pase [esta prueba](/edit/sesión8/bloomfilter.py)

In [ ]:
# Ejercicio 1.1
from bloomfilter import BloomFilter, CONSTANTE_BIP37
from ayudante import campo_bit_a_bytes, murmur3

tamaño_campo = 10
función_contador = 5
ajuste = 99
cosas = (b'Hello World',  b'Goodbye!')

# tamaño_campo_bit is 8 * tamaño_campo
tamaño_campo_bit = tamaño_campo * 8
# crea un campo bit con el tamaño apropiado
campo_bit = [0] * tamaño_campo_bit

# para cada cosa deberás añadir el filtro
for cosa in cosas:
    # itera tantas veces como valga función_contador
    for i in range(función_contador):
        # La semilla BIP0037 spec es i*CONSTANTE_BIP37 + ajuste
        semilla = i * CONSTANTE_BIP37 + ajuste
        # obtén el hash murmur3 dada esa semilla
        h = murmur3(cosa, semilla=semilla)
        # establece el bit para ser h módulo el tamaño_campo_bit
        bit = h % tamaño_campo_bit
        # establece el campo_bit en el bit index para ser 1
        campo_bit[bit] = 1
# imprime el campo bit convertido a bytes usando campo_bit_a_bytes en hex
print(campo_bit_a_bytes(campo_bit).hex())

4000600a080000010940


In [ ]:
# Ejercicio 1.2

reload(bloomfilter)
ejecutar_prueba(bloomfilter.BloomFilterTest('prueba_sumar'))

.
----------------------------------------------------------------------
Prueba ejecutada en 0.001s

OK


### Ejercicio 2

#### 2.1. Haz que pase [esta prueba](/edit/sesión8/bloomfilter.py)

#### 2.2. Haz lo siguiente:

* Conéctate a un nodo testnet
* Carga un filtro para tu dirección testnet
* Envía una solicitud para transacciones desde el bloque que tenía tu transacción testnet previa
* Recibe el bloquemerkle y mensajes tx.

In [ ]:
# Ejercicio 2.1

reload(bloomfilter)
ejecutar_prueba(bloomfilter.BloomFilterTest('prueba_filtrocarga'))

.
----------------------------------------------------------------------
Prueba ejecutada en 0.001s

OK


In [ ]:
# Ejercicio 2.2

from bloomfilter import BloomFilter
from cce import ClavePrivada
from ayudante import decodificar_base58, doble_sha256, little_endian_a_int
from bloquemerkle import BloqueMerkle
from network import (
    TIPO_DATOS_BLOQUE_FILTRADO, 
    ObtenerMensajeDatos,
    NodoSimple,
)

hash_bloque = bytes.fromhex('00000000000001fd16e986436252c023b2f9ba729319309189af9ab5be9d4ff9')
contraseña = b'jimmy@programmingbloquechain.com secret passphrase'  # RELLENA ESTO
secreto = little_endian_a_int(doble_sha256(contraseña))
clave_privada = ClavePrivada(secreto=secreto)
dire = clave_privada.punto.dire(testnet=True)
print(dire)
tamaño_filtro = 30
num_funciones_filtro = 5
ajuste_filtro = 90210  # RELLENA ESTO

# obtén el hash160 de la dirección, que es lo que necesitamos añadir a nuestro filtro
h160 = decodificar_base58(dire)
# crear un filtro bloom de tamaño 30 y 5 funciones. Añade el ajuste que quieras
bf = BloomFilter(tamaño_filtro, num_funciones_filtro, ajuste_filtro)
# añade el h160 al filtro bloom
bf.add(h160)

# conéctate a tbtc.programmingbloquechain.com en modo testnet, logging True
node = NodoSimple('tbtc.programmingbloquechain.com', testnet=True, logging=True)
# completa el handshake
node.handshake()
# carga el filtro bloom con el comando filtrocarga
node.send(b'filterload', bf.filterload())
# crea un mensaje obtenerdatos
obtenerdatos = ObtenerMensajeDatos()
# añade un TIPO_DATOS_BLOQUE_FILTRADO para el hash de bloque arriba
obtenerdatos.agregar_datos(TIPO_DATOS_BLOQUE_FILTRADO, hash_bloque)
# enviar un mensaje obtener datos para un TIPO_DATOS_BLOQUE_FILTRADO
node.send(obtenerdatos.comando, obtenerdatos.serializar())
# espera por el comando bloquemerkle
envelope = node.esperar_comandos([b'bloquemerkle'])
print(envelope)
# esperar por el comando tx
envelope = node.esperar_comandos([b'tx'])
print(envelope)

mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv
enviando: version: 7f11010000000000000000000dc0bf5b00000000000000000000000000000000000000000000ffff000000008d20000000000000000000000000000000000000ffff000000008d208d07ece835fe28291b2f70726f6772616d6d696e67626c6f636b636861696e3a302e312f0000000001
recibiendo: version: 7f1101000d040000000000000ec0bf5b00000000000000000000000000000000000000000000ffff4830fd33cf700d04000000000000000000000000000000000000000000000000f555f8496387e698102f5361746f7368693a302e31372e302f79e9150001
enviando: verack: 
recibiendo: verack: 
enviando: filterload: 1e000000000000008800000002000000000000000000200000000100000000050000006260010001
enviando: obtenerdatos: 0103000000f94f9dbeb59aaf899130199372baf9b223c052624386e916fd01000000000000
recibiendo: sendheaders: 
recibiendo: sendcmpct: 000200000000000000
recibiendo: sendcmpct: 000100000000000000
recibiendo: ping: 48215b048bb8daf4
enviando: pong: 48215b048bb8daf4
recibiendo: dire: 010ec0bf5b0d0400000000000000000000000000000000ffffa2d49

### Ejercicio 3

#### 3.1. Haz que pase [esta prueba](/edit/sesión8/bloquemerkle.py)

In [ ]:
# Ejercicio 3.1

reload(bloquemerkle)
ejecutar_prueba(bloquemerkle.PruebaBloqueMerkle('prueba_es_válida'))

.
----------------------------------------------------------------------
Prueba ejecutada en 0.003s

OK


### Ejercicio 4

#### 4.1. Se te han enviado una cantidad indeterminada de bitcoins testnet a tu dirección

Envíalos todos de vuelta (menos comisiones) a `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv` usando solo el protocolo de red

In [ ]:
# Ejercicio 4.1

import time

from bloque import Block
from bloomfilter import BloomFilter
from cce import ClavePrivada
from ayudante import (
    decodificar_base58,
    doble_sha256,
    codificar_varint,
    little_endian_a_int, 
    read_varint,
    SIGHASH_ALL,
)
from bloquemerkle import BloqueMerkle
from network import (
    ObtenerMensajeDatos,
    ObtenerMensajeCabeceras,
    MensajeCabeceras,
    NetworkEnvelope,
    NodoSimple,
    TIPO_DATOS_TX,
    TIPO_DATOS_BLOQUE_FILTRADO,
)
from script import script_p2pkh
from tx import Tx, TxIn, TxOut

hex_último_bloque = '0000000000000b8e8529996b47b8c3dd13c70adfaf94278883e05bba6ae9159a'
contraseña = b'jimmy@programmingbloquechain.com secreto passphrase'  # RELLENA ESTO
secreto = little_endian_a_int(doble_sha256(contraseña))
clave_privada = ClavePrivada(secreto=secreto)
dire = clave_privada.punto.dire(testnet=True)
print(dire)
tamaño_filtro = 30
num_funciones_filtro = 5
ajuste_filtro = 90210  # RELLENA ESTO

dirección_destino = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
target_h160 = decodificar_base58(dirección_destino)
script_destino = script_p2pkh(target_h160)
comisión = 5000  # comisión en satoshis

# obtén el hash160 de la dirección, que es lo que necesitamos añadir a nuestro filtro
h160 = decodificar_base58(dire)
# crea un bloom filter de tamaño 30 y 5 funciones. Añade el ajuste que quieras
bf = BloomFilter(tamaño_filtro, num_funciones_filtro, ajuste_filtro)
# añade el h160 al filtro bloom
bf.add(h160)

# conéctate a tbtc.programmingbloquechain.com en modo testnet, logging True
node = NodoSimple('tbtc.programmingbloquechain.com', testnet=True, logging=True)
# completa el handshake
node.handshake()
# carga el filtro bloom con el comando filterload
node.send(b'filterload', bf.filterload())

# establece bloque_inicial a bloque_final desde arriba
bloque_inicial = bytes.fromhex(hex_último_bloque)
# crea ObtenerMensajeCabeceras con este bloque inicial
ghm = ObtenerMensajeCabeceras(bloque_inicial=bloque_inicial)
# enviar un mensaje obtenercabeceras con el bloque_inicial
node.send(ghm.comando, ghm.serializar())

# esperar por el mensaje cabeceras
envelope = node.esperar_comandos([MensajeCabeceras.comando])

# obtén el stream desde las cabeceras
s = envelope.stream()
# parsea el mensaje de cabeceras
mensaje_cabeceras = MensajeCabeceras.parsear(s)
# guarda el último bloque como None
último_bloque = None
# inicializa ObtenerMensajeDatos
obtenerdatos = ObtenerMensajeDatos()
# haz un bucle a través de los bloques en las cabeceras
for bloque in mensaje_cabeceras.bloques:
    # comprueba que la prueba de trabajo en el bloque sea válida
    if not bloque.comprobar_pow():
        raise RuntimeError
    # comprueba que el bloque_previo de este bloque sea el último bloque
    if último_bloque is not None and bloque.bloque_previo != último_bloque:
        raise RuntimeError
    # agrega TIPO_DATOS_BLOQUE_FILTRADO bloque.hash() a ObtenerMensajeDatos
    obtenerdatos.agregar_datos(TIPO_DATOS_BLOQUE_FILTRADO, bloque.hash())
    # establece el último bloque al hash actual
    último_bloque = bloque.hash()
# envía el mensaje obtenerdatos
node.send(obtenerdatos.comando, obtenerdatos.serializar())

# inicializa tx_previa e índice_previo a None
tx_previa, índice_previo = None, None
# inicializa cantidad_previa a None
cantidad_previa = None
# while tx_previa is None 
while tx_previa is None:
    # espera por el bloquemerkle o por los comandos tx
    envelope = node.esperar_comandos([b'tx', b'bloquemerkle'])
    # inicializa el stream desde el envelope
    s = envelope.stream()
    # si tenemos el comando bloquemerkle
    if envelope.comando == b'bloquemerkle':
        # parsea el BloqueMerkle
        mb = BloqueMerkle.parsear(s)
        # comprueba que el BloqueMerkle es válido
        if not mb.es_válido():
            raise RuntimeError
    # en caso contrario tenemos un comando tx
    else:
        # parsea la tx
        t = Tx.parsear(s)
        # haz un bucle a través de los tx outs enumerados (enumerate(t.tx_outs))
        for i, tx_out in enumerate(t.tx_outs):
            print(tx_out)
            # si nuestro output tiene la misma dirección que la nuestra (dire), lo hemos encontrado
            if tx_out.script_pubkey.direess(testnet=True) == dire:
                # hemos encontrado nuestra utxo. establece tx_previa, índice_previo, cantidad_previa
                tx_previa = t.hash()
                índice_previo = i
                cantidad_previa = tx_out.cantidad
                # establece el cache para queno tengamos que mirar la tx en un block explorer
                # Use: TxIn.set_cache(tx_previa, t.serializar())
                TxIn.set_cache(tx_previa, t.serializar())
# crea tx_in
tx_in = TxIn(tx_previa, i, b'', 0xffffffff)
# calcula la cantidad output (cantidad_previa - comisión)
output_cantidad = cantidad_previa - comisión
# crea una nueva TxOut a script_destino.serializar() con el output cantidad
tx_out = TxOut(output_cantidad, script_destino)
# crea una nueva transacción (testnet=True)
t = Tx(1, [tx_in], [tx_out], 0, testnet=True)
# firma la transacción en input 0 con tu clave privada y SIGHASH_ALL
t.firmar_input(0, clave_privada, SIGHASH_ALL)
# serializar y hex para ver a qué se parece
print(t.serializar().hex())
# envía esta transacción firmada a la red
node.send(b'tx', t.serializar())
# espera un segundo para que este mensaje pase al otro nodo time.sleep(1) 
time.sleep(1)
# ahora pregunta por esta transacción al otro nodo
# crea un ObtenerMensajeDatos
obtenerdatos = ObtenerMensajeDatos()
# pregunta por nuestra transacción añadiendo su hash y TX_DATATYPE al mensaje
obtenerdatos.agregar_datos(TIPO_DATOS_TX, t.hash())
# envía el mensaje
node.send(obtenerdatos.comando, obtenerdatos.serializar())
# ahora espera a una respuesta
envelope = node.esperar_comandos({b'tx', b'reject'})
if envelope.comando == b'tx':
    print(envelope.carga.hex())
    print(t.hash().hex())
else:
    print(envelope.carga)

mqYz6JpuKukHzPg94y4XNDdPCEJrNkLQcv
enviando: version: 7f110100000000000000000018c0bf5b00000000000000000000000000000000000000000000ffff000000008d20000000000000000000000000000000000000ffff000000008d20c728f0203ebb2bd31b2f70726f6772616d6d696e67626c6f636b636861696e3a302e312f0000000001
recibiendo: version: 7f1101000d0400000000000019c0bf5b00000000000000000000000000000000000000000000ffff4830fd33cf800d04000000000000000000000000000000000000000000000000177d83cab5388f1e102f5361746f7368693a302e31372e302f79e9150001
enviando: verack: 
recibiendo: verack: 
enviando: filterload: 1e000000000000008800000002000000000000000000200000000100000000050000006260010001
enviando: getheaders: 7f110100019a15e96aba5be083882794afdf0ac713ddc3b8476b9929858e0b0000000000000000000000000000000000000000000000000000000000000000000000000000
recibiendo: sendheaders: 
recibiendo: sendcmpct: 000200000000000000
recibiendo: sendcmpct: 000100000000000000
recibiendo: ping: efcaa67263a33e3a
enviando: pong: efcaa67263a33e3a
recibiendo: